In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import ast
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')
movies.head()
credits.head()
movies=movies.merge(credits,on='title')
movies = movies[['movie_id','title','genres','keywords','overview','cast','crew']]
movies.isnull().sum()      #checks missing data so we removed it
movies.dropna(inplace=True)  # we r dropping  that rows here
movies.duplicated().sum()   # we r  checking for the duplicate data as well


def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):    #to convert the list of dic in list whi is given as string in data
        l.append(i['name'])            #converting into a list which only have the words without any numbers
        
    return l

movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

def convert3(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):  #it takes the name of first three elements of the list 
        if counter!=3:
            l.append(i['name'])
            counter=counter+1
        else:
            break
    return l


movies['cast']=movies['cast'].apply(convert3)    #taking first three actor of dictionary value whose key is name

def fetch_dir(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):          #it fetches director in the crew  whose job is director 
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

movies['crew']=movies['crew'].apply(fetch_dir)
movies['overview']=movies['overview'].apply(lambda x:x.split())   #splitting the line in words

movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [3]:

movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']



In [ ]:
movies.head()

In [ ]:
new_df=movies[['movie_id','title','tags']]
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(new_df['tags']).toarray()
vectors
feature_names = list(cv.vocabulary_.keys())
feature_names

In [ ]:
new_df.head()

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
similarity= cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True ,key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True ,key=lambda x:x[1])[1:6]
    for i in movies_list:
       print(new_df.iloc[i[0]]['title'])

recommend('Avatar')
    

In [ ]:
recommend('Avatar')

In [ ]:
new_df['tags'][0]

In [14]:
import pickle
pickle.dump(new_df,open('movies.pkl','wb'))
#pickle.dump(new_df.to_dict,open('movies_dict.pkl','wb'))

In [ ]:
new_df['title'].values

In [16]:
pickle.dump(similarity,open('similarity.pkl','wb'))